
Case 1 - drop

Case 2 - Ordinal encoding (Order of categories(Good,better,best))

Case 3 - One-hot encoding (Categories as binary variables)

In [27]:
#import modin.pandas as pd
from pandas_profiling import ProfileReport
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import (GridSearchCV, StratifiedKFold,
                                     train_test_split)
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, mean_absolute_error,
                             roc_auc_score)
from sklearn.impute import SimpleImputer
from sklearn.ensemble import (AdaBoostClassifier, RandomForestClassifier,
                              RandomForestRegressor)
import xgboost as xgb
import scipy.stats as stats
import requests
import pyforest
import cv2 as cv
import bs4 as bs
import warnings
import time
import re
import random
import os
import math
import datetime
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline
# from autoviz.AutoViz_Class import AutoViz_Class

%config InlineBackend.figure_format = 'retina'

#import keras as ks
# import tensorflow as tf

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
sns.set(style='whitegrid', color_codes=True, font_scale=1.5,
        palette='plasma', font='Arial', rc={'figure.figsize': (8, 6)})


In [28]:
df=pd.read_csv('./data/melb_data.csv')
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,85 Turner St,2,h,1480000.00,S,Biggin,3/12/2016,2.50,3067.00,2.00,1.00,1.00,202.00,NaN,NaN,Yarra,-37.80,145.00,Northern Metropolitan,4019.00
1,Abbotsford,25 Bloomburg St,2,h,1035000.00,S,Biggin,4/02/2016,2.50,3067.00,2.00,1.00,0.00,156.00,79.00,1900.00,Yarra,-37.81,144.99,Northern Metropolitan,4019.00
2,Abbotsford,5 Charles St,3,h,1465000.00,SP,Biggin,4/03/2017,2.50,3067.00,3.00,2.00,0.00,134.00,150.00,1900.00,Yarra,-37.81,144.99,Northern Metropolitan,4019.00
3,Abbotsford,40 Federation La,3,h,850000.00,PI,Biggin,4/03/2017,2.50,3067.00,3.00,2.00,1.00,94.00,NaN,NaN,Yarra,-37.80,145.00,Northern Metropolitan,4019.00
4,Abbotsford,55a Park St,4,h,1600000.00,VB,Nelson,4/06/2016,2.50,3067.00,3.00,1.00,2.00,120.00,142.00,2014.00,Yarra,-37.81,144.99,Northern Metropolitan,4019.00


# 1. Drop missing values

In [29]:
X=df.drop(['Price'],axis=1)
y=df['Price']

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [35]:
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()] 
X_train.drop(cols_with_missing, axis=1, inplace=True)
X_test.drop(cols_with_missing, axis=1, inplace=True)
cols_with_missing

[]

In [34]:
low_cardinality_cols = [cname for cname in X_train.columns if X_train[cname].nunique() < 10 and 
                        X_train[cname].dtype == "object"]
low_cardinality_cols

['Type', 'Method', 'Regionname']

In [37]:
# Select numerical columns
numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]
numerical_cols

['Rooms',
 'Distance',
 'Postcode',
 'Bedroom2',
 'Bathroom',
 'Landsize',
 'Lattitude',
 'Longtitude',
 'Propertycount']

In [42]:
my_cols = low_cardinality_cols + numerical_cols
my_cols
X_train = X_train[my_cols].copy()
X_test = X_test[my_cols].copy()
X_train, X_test

['Type',
 'Method',
 'Regionname',
 'Rooms',
 'Distance',
 'Postcode',
 'Bedroom2',
 'Bathroom',
 'Landsize',
 'Lattitude',
 'Longtitude',
 'Propertycount']

(      Type Method                  Regionname  Rooms  Distance  Postcode  \
 4055     h      S        Western Metropolitan      5      6.90   3039.00   
 10534    h     PI        Eastern Metropolitan      5     12.40   3107.00   
 12218    h     PI        Western Metropolitan      3      9.40   3025.00   
 7300     h      S        Western Metropolitan      2     13.90   3020.00   
 12412    t     VB       Northern Metropolitan      3      7.00   3071.00   
 ...    ...    ...                         ...    ...       ...       ...   
 12890    h     SP        Western Metropolitan      3     12.80   3022.00   
 5291     h     SP       Northern Metropolitan      3      2.60   3121.00   
 9978     h     SP  South-Eastern Metropolitan      3     34.10   3805.00   
 11639    h      S       Northern Metropolitan      4     20.60   3064.00   
 2699     u     PI        Western Metropolitan      1      6.40   3011.00   
 
        Bedroom2  Bathroom  Landsize  Lattitude  Longtitude  Propertycount

In [43]:
# Get list of categorical variables
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['Type', 'Method', 'Regionname']


In [44]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

CASE 1 - dropping missing values

In [47]:
drop_X_train = X_train.select_dtypes(exclude=['object'])
drop_X_test = X_test.select_dtypes(exclude=['object'])

print("MAE from Approach 1 (Drop categorical variables):")
print(score_dataset(drop_X_train, drop_X_test, y_train, y_test))

MAE from Approach 1 (Drop categorical variables):
183133.62855849814


# CASE 2 - Ordinal encoding

In [49]:
# Make copy to avoid changing original data 
label_X_train = X_train.copy()
label_X_valid = X_test.copy()

In [50]:
# Apply ordinal encoder to each column with categorical data
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
label_X_train[object_cols] = ordinal_encoder.fit_transform(X_train[object_cols])
label_X_valid[object_cols] = ordinal_encoder.transform(X_test[object_cols])

In [51]:
label_X_train

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Landsize,Lattitude,Longtitude,Propertycount
4055,0.00,1.00,6.00,5,6.90,3039.00,5.00,1.00,584.00,-37.77,144.92,6232.00
10534,0.00,0.00,0.00,5,12.40,3107.00,5.00,2.00,665.00,-37.77,145.12,5420.00
12218,0.00,0.00,6.00,3,9.40,3025.00,3.00,1.00,693.00,-37.84,144.85,5132.00
7300,0.00,1.00,6.00,2,13.90,3020.00,2.00,1.00,497.00,-37.78,144.82,2185.00
12412,1.00,4.00,2.00,3,7.00,3071.00,3.00,2.00,120.00,-37.76,145.02,8870.00
...,...,...,...,...,...,...,...,...,...,...,...,...
12890,0.00,3.00,6.00,3,12.80,3022.00,3.00,2.00,739.00,-37.77,144.80,1281.00
5291,0.00,3.00,2.00,3,2.60,3121.00,3.00,1.00,238.00,-37.82,145.00,14949.00
9978,0.00,3.00,4.00,3,34.10,3805.00,3.00,1.00,810.00,-38.03,145.31,9376.00
11639,0.00,1.00,2.00,4,20.60,3064.00,4.00,2.00,420.00,-37.59,144.90,15510.00


# CASE 3 - One-hot encoding

In [52]:
from sklearn.preprocessing import OneHotEncoder

# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_test[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_test.index

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_test = X_test.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
OH_X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
OH_X_test = pd.concat([num_X_test, OH_cols_valid], axis=1)

print("MAE from Approach 3 (One-Hot Encoding):") 
print(score_dataset(OH_X_train, OH_X_test, y_train, y_test))

MAE from Approach 3 (One-Hot Encoding):
172000.09068044744
